In [1]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.io
import scipy
from scipy import signal

#from statsmodels.tsa.holtwinters  import SimpleExpSmoothing, Holt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.manifold import Isomap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture as GMM
from sklearn import preprocessing

# Feature extraction

In [2]:
df_butter=pd.read_csv('dataset_processed.csv')

In [3]:
df_butter.head()

,0,1,2,3,4,5,6,7,8,9,...,5990,5991,5992,5993,5994,5995,5996,5997,5998,5999
0,0.543521,0.536693,0.529900,0.523143,0.516427,0.509753,0.503124,0.496543,0.490013,0.483537,...,0.367887,0.367886,0.367886,0.367886,0.367886,0.367886,0.367886,0.367886,0.367886,0.367886
1,0.458166,0.456752,0.455336,0.453918,0.452499,0.451079,0.449659,0.448240,0.446821,0.445404,...,0.392877,0.392877,0.392878,0.392878,0.392878,0.392879,0.392879,0.392879,0.392879,0.392879
2,0.938540,0.948536,0.958461,0.968311,0.978083,0.987773,0.997377,1.006894,1.016319,1.025650,...,0.554424,0.554424,0.554425,0.554425,0.554425,0.554425,0.554425,0.554426,0.554426,0.554426
3,0.117722,0.118147,0.118589,0.119049,0.119526,0.120020,0.120531,0.121058,0.121603,0.122163,...,0.573146,0.573142,0.573138,0.573135,0.573132,0.573130,0.573128,0.573126,0.573125,0.573124
4,0.442304,0.437091,0.431882,0.426682,0.421490,0.416311,0.411145,0.405996,0.400865,0.395756,...,0.358348,0.358348,0.358347,0.358347,0.358346,0.358346,0.358346,0.358346,0.358345,0.358345


In [4]:
from feature_extraction import *
frame=6000
step=1
signal=df_butter.to_numpy()


variance = []
rms = []
iemg = []
mav = []
log_detector = []
wl = []
aac = []
dasdv = []
zc = []
wamp = []
myop = []

th = np.mean(signal[0]) + 3 * np.std(signal[0])


x = signal[0]

#variance.append(np.var(x))
rms.append(np.sqrt(np.mean(x ** 2)))
iemg.append(np.sum(abs(x)))  # Integral
mav.append(np.sum(np.absolute(x)) )  # Mean Absolute Value
wl.append(np.sum(abs(np.diff(x))))  # Wavelength
aac.append(np.sum(abs(np.diff(x))) )  # Average Amplitude Change
dasdv.append(
    math.sqrt( np.sum((np.diff(x)) ** 2)))  # Difference absolute standard deviation value
#zc.append(zcruce(x, th))  # Zero-Crossing
wamp.append(wilson_amplitude(x, th))  # Willison amplitude
myop.append(myopulse(x, th))  # Myopulse percentage rate

x_input= np.column_stack(( rms, iemg, mav, wl, aac, dasdv, wamp, myop))
for i in range(len(signal)-1):
    

    variance = []
    rms = []
    iemg = []
    mav = []
    log_detector = []
    wl = []
    aac = []
    dasdv = []
    zc = []
    wamp = []
    myop = []

    th = np.mean(signal[i]) + 3 * np.std(signal[i])


    x = signal[i]

    #variance.append(np.var(x))
    rms.append(np.sqrt(np.mean(x ** 2)))
    iemg.append(np.sum(abs(x)))  # Integral
    mav.append(np.sum(np.absolute(x)) )  # Mean Absolute Value
    wl.append(np.sum(abs(np.diff(x))))  # Wavelength
    aac.append(np.sum(abs(np.diff(x))) )  # Average Amplitude Change
    dasdv.append(
        math.sqrt( np.sum((np.diff(x)) ** 2)))  # Difference absolute standard deviation value
    #zc.append(zcruce(x, th))  # Zero-Crossing
    wamp.append(wilson_amplitude(x, th))  # Willison amplitude
    myop.append(myopulse(x, th))  # Myopulse percentage rate

    tt= np.column_stack(( rms, iemg, mav, wl, aac, dasdv, wamp, myop))
    x_input=np.row_stack((x_input,tt))


In [5]:
x_input.shape

(900, 8)

In [37]:
dataset=pd.read_csv('data.csv')
df_labels = dataset.iloc[:, 6001]
label_dict = {
    'Spherical':1,
    'Tip':2,
    'Palmar':3,
    'Lateral':4,
    'Cylindrical':5,
    'Hook':6}

col = [label_dict[i] for i in df_labels.values.tolist()]

In [40]:
#PCA Transform
n = 8
pca = PCA(n_components=n).fit(df_butter.T)
df_pca = pca.components_.T

#Isomap Transform
iso = Isomap(n_components=n)
df_iso = iso.fit_transform(df_butter)

## Classification Models

Six classification models were used to classify the data. We found no difference in the PCA and Isomap dataframe accuracies, so for all of the models we used "df_iso" as our final dataset. Here are the classification models used.

* Support Vector Machine 
* Logistic Regression
* Neural Net
* K-Means
* Gaussian Mixture
* Naive-Bayes

All hyper-parameters were tuned through grid search. The following models show the optimal hyperparameters. 

In [46]:
svm_res = []
log_res = []
nn_res = []
nb_res = []
kmeans_res = []
gmm_res = []

le = preprocessing.LabelEncoder()
le.fit(y_train)

for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(x_input, 
                                                        df_labels, 
                                                        test_size=0.2, 
                                                        random_state=i)

    clf_svc = make_pipeline(StandardScaler(), SVC(gamma='auto', C=10))
    clf_log = make_pipeline(StandardScaler(), LogisticRegression(C=10, max_iter=1000))
    clf_nn = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(200,200,200), max_iter=2000))
    clf_nb = make_pipeline(StandardScaler(), GaussianNB())
    clf_kmeans = make_pipeline(StandardScaler(), KMeans(n_clusters=6))
    clf_gmm = make_pipeline(StandardScaler(), GMM(n_components=6))
    
    clfs = [clf_svc, 
            clf_log, 
            clf_nn, 
            clf_nb, 
            clf_kmeans, 
            clf_gmm]
    
    for c in clfs:
        c.fit(X_train, y_train)

    svm_labels = clf_svc.predict(X_test)
    log_labels = clf_log.predict(X_test)
    nn_labels = clf_nn.predict(X_test)
    nb_labels = clf_nb.predict(X_test)
    kmeans_labels = clf_kmeans.predict(X_test)
    gmm_labels = clf_gmm.predict(X_test)

    svm_res.append(((svm_labels == y_test).value_counts()/len(X_test))[1])
    log_res.append(((log_labels == y_test).value_counts()/len(X_test))[1])
    nn_res.append(((nn_labels == y_test).value_counts()/len(X_test))[1])
    nb_res.append(((nb_labels == y_test).value_counts()/len(X_test))[1])
    kmeans_res.append(1 - (sum([abs(i[0] - i[1]) for i in zip(sorted(np.bincount(le.transform(y_test))), 
                                       sorted(np.bincount(kmeans_labels)))])/len(y_test)))
    gmm_res.append(1 - (sum([abs(i[0] - i[1]) for i in zip(sorted(np.bincount(le.transform(y_test))), 
                                       sorted(np.bincount(gmm_labels)))])/len(y_test)))

svm_score = np.max(np.max(np.array(svm_res)))
log_score = np.max(np.max(np.array(log_res)))
nn_score = np.max(np.max(np.array(nn_res)))
nb_score = np.max(np.max(np.array(nb_res)))
kmeans_score = np.max(np.max(np.array(kmeans_res)))
gmm_score = np.max(np.max(np.array(gmm_res)))

In [47]:
print("SVM Accuracy: ", svm_score)
print("LR Accuracy: ", log_score)
print("Neural Network accuracy:", nn_score)
print("Naive Bayes accuracy", nb_score)
print("K-Means accuracy:", kmeans_score)
print("GMM accuracy", gmm_score)

SVM Accuracy:  0.5555555555555556
LR Accuracy:  0.49444444444444446
Neural Network accuracy: 0.6555555555555556
Naive Bayes accuracy 0.4666666666666667
K-Means accuracy: 0.6555555555555556
GMM accuracy 0.8333333333333334
